In [1]:
class Opion():
    
    def __init__(self):
        
        self.dataroot= r'/home/jara/DeepInPainting/paris_original_test' #image dataroot
        self.maskroot= r'/home/jara/DeepInPainting/mask/test_mask'
        self.batchSize= 1   # Need to be set to 1
        self.fineSize=256 # image size
        self.input_nc=3  # input channel size for first stage
        self.input_nc_g=6 # input channel size for second stage
        self.output_nc=3# output channel size
        self.ngf=64 # inner channel
        self.ndf=64# inner channel
        self.which_model_netD='basic' # patch discriminator
        self.which_model_netF='feature'# feature patch discriminator
        self.which_model_netG='unet_csa'# second stage network
        self.which_model_netP='unet_256'# first stage network
        self.triple_weight=1
        self.name='CSA_inpainting'
        self.n_layers_D='3' # network depth
        self.gpu_ids=[0]
        self.model='csa_net'
        self.checkpoints_dir=r'/home/jara/DeepInPainting/checkpoints' #
        self.norm='instance'
        self.fixed_mask=1
        self.use_dropout=False
        self.init_type='normal'
        self.mask_type='random'
        self.lambda_A=100
        self.threshold=5/16.0
        self.stride=1
        self.shift_sz=1 # size of feature patch
        self.mask_thred=1
        self.bottleneck=512
        self.gp_lambda=10.0
        self.ncritic=5
        self.constrain='MSE'
        self.strength=1
        self.init_gain=0.02
        self.cosis=1
        self.gan_type='lsgan'
        self.gan_weight=0.2
        self.overlap=4
        self.skip=0
        self.display_freq=1000
        self.print_freq=50
        self.save_latest_freq=5000
        self.save_epoch_freq=2
        self.continue_train=False
        self.epoch_count=1
        self.phase='train'
        self.which_epoch=60
        self.niter=20
        self.niter_decay=100
        self.beta1=0.5
        self.lr=0.0002
        self.lr_policy='lambda'
        self.lr_decay_iters=50
        self.isTrain=False

In [2]:
import time
from util.data_load import Data_load
from models.models import create_model
import torch
import os
import torchvision
from torch.utils import data
import torchvision.transforms as transforms
from IQA_pytorch import SSIM
import numpy as np
opt = Opion()
transform_mask = transforms.Compose(
    [transforms.Resize((opt.fineSize,opt.fineSize)),
     transforms.ToTensor(),
    ])
transform = transforms.Compose(
    [transforms.Resize((opt.fineSize,opt.fineSize)),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.5] * 3, std=[0.5] * 3)])
transform_ref = transforms.Compose(
    [transforms.RandomResizedCrop((opt.fineSize,opt.fineSize),scale=(0.8,1.0),ratio=(1,1)),
     transforms.ColorJitter(brightness=0.1,contrast=0.1,saturation=0.1,hue=0.1),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.5] * 3, std=[0.5] * 3)])

dataset_test = Data_load(opt.dataroot, opt.maskroot,opt.dataroot, transform, transform_mask,transform_ref)
iterator_test = (data.DataLoader(dataset_test, batch_size=opt.batchSize,shuffle=False))
print(len(dataset_test))
model = create_model(opt)
total_steps = 0


2400
csa_net
initialize network with normal


/home/jara/DeepInPainting/models/networks.py:54: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  init.normal(m.weight.data, 0.0, gain)
/home/jara/DeepInPainting/models/networks.py:64: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(m.bias.data, 0.0)


initialize network with normal
Loading pre-trained network!
model [CSAModel] was created


In [3]:
load_epoch=60
model.load(load_epoch)

In [4]:
save_dir = r'/home/jara/DeepInPainting/result'
if os.path.exists(save_dir) is False:
    os.makedirs(save_dir)

epoch=1
i=0
psnr_average, ssim_average = 0, 0
psnr_sum, ssim_sum = 0, 0
for image,mask,ref in (iterator_test):
    
    if epoch > 500:
        psnr_average = psnr_sum / 500
        ssim_average = ssim_sum / 500
        print("Finish")
        print('PSNR_average : %.2f, SSIM_average : %.3f' %(psnr_average, ssim_average))
        break

    iter_start_time = time.time()
    image=image.cuda()
    mask=mask.cuda()
    mask=mask[0][0]
    mask=torch.unsqueeze(mask,0)
    mask=torch.unsqueeze(mask,1)
    mask=mask.bool()

    model.set_input(image,mask,image) # it not only sets the input data with mask, but also sets the latent mask.
    model.set_ref_latent()
    model.set_gt_latent()
    model.test()
    
    real_A,ref_B,fake_B,fake_P,real_B=model.get_current_visuals()
    pic = (torch.cat([real_A,ref_B,fake_P,fake_B], dim=0) + 1) / 2.0
    
    torchvision.utils.save_image(pic, '%s/Epoch_(%d)_(%dof%d).jpg' % (
    save_dir, epoch, total_steps + 1, len(dataset_test)), nrow=2)
    
    psnr=0
    mse=0
    if type(real_B)==torch.Tensor:
        mse=torch.mean((real_B-fake_B)**2)
    else:
        mse=np.mean((real_B-fake_B)**2)
    
    if mse==0:
        psnr=100
    else:
        psnr=10*torch.log10((2**2)/mse)
    errors=model.get_error()
    
    mod=SSIM(channels=3)
    score=mod(real_B,fake_B,as_loss=False)
    print('%d. PSNR : %f, SSIM : %f' %(epoch,psnr,score))
    psnr_sum += psnr
    ssim_sum += score
    epoch=epoch+1


1. PSNR : 30.119421, SSIM : 0.912544
2. PSNR : 35.014584, SSIM : 0.929447
3. PSNR : 23.449467, SSIM : 0.768153
4. PSNR : 18.309118, SSIM : 0.475725
5. PSNR : 37.821716, SSIM : 0.980185
6. PSNR : 26.166348, SSIM : 0.827192
7. PSNR : 16.317219, SSIM : 0.549187
8. PSNR : 30.598316, SSIM : 0.948930
9. PSNR : 31.358784, SSIM : 0.901294
10. PSNR : 38.279476, SSIM : 0.980753
11. PSNR : 22.636759, SSIM : 0.682630
12. PSNR : 21.924118, SSIM : 0.649245
13. PSNR : 26.198479, SSIM : 0.870930
14. PSNR : 23.715351, SSIM : 0.785881
15. PSNR : 25.349216, SSIM : 0.814099
16. PSNR : 24.616335, SSIM : 0.698449
17. PSNR : 23.085478, SSIM : 0.705646
18. PSNR : 27.996445, SSIM : 0.878576
19. PSNR : 21.806404, SSIM : 0.762748
20. PSNR : 15.964593, SSIM : 0.569104
21. PSNR : 24.297443, SSIM : 0.836321
22. PSNR : 29.041504, SSIM : 0.837091
23. PSNR : 21.597651, SSIM : 0.738494
24. PSNR : 25.384663, SSIM : 0.769015
25. PSNR : 32.509605, SSIM : 0.926549
26. PSNR : 26.658199, SSIM : 0.812871
27. PSNR : 26.271461,

214. PSNR : 17.598173, SSIM : 0.519937
215. PSNR : 30.494247, SSIM : 0.885044
216. PSNR : 18.355473, SSIM : 0.566773
217. PSNR : 37.928688, SSIM : 0.907559
218. PSNR : 41.539009, SSIM : 0.989374
219. PSNR : 21.364288, SSIM : 0.718666
220. PSNR : 33.233818, SSIM : 0.955146
221. PSNR : 25.632641, SSIM : 0.876658
222. PSNR : 30.676813, SSIM : 0.975900
223. PSNR : 16.974483, SSIM : 0.548842
224. PSNR : 22.532293, SSIM : 0.709217
225. PSNR : 22.234337, SSIM : 0.771240
226. PSNR : 23.810444, SSIM : 0.705598
227. PSNR : 26.434155, SSIM : 0.866107
228. PSNR : 29.313530, SSIM : 0.937854
229. PSNR : 17.082718, SSIM : 0.402877
230. PSNR : 20.319633, SSIM : 0.732713
231. PSNR : 29.413265, SSIM : 0.837656
232. PSNR : 23.051641, SSIM : 0.812781
233. PSNR : 25.964252, SSIM : 0.844095
234. PSNR : 26.852917, SSIM : 0.936463
235. PSNR : 19.760086, SSIM : 0.680841
236. PSNR : 31.609642, SSIM : 0.927821
237. PSNR : 23.647396, SSIM : 0.812776
238. PSNR : 41.429977, SSIM : 0.975263
239. PSNR : 25.819069, SS

425. PSNR : 20.883333, SSIM : 0.737400
426. PSNR : 22.826624, SSIM : 0.648846
427. PSNR : 20.828894, SSIM : 0.799983
428. PSNR : 22.053841, SSIM : 0.649861
429. PSNR : 36.810425, SSIM : 0.971430
430. PSNR : 22.963135, SSIM : 0.755553
431. PSNR : 24.559595, SSIM : 0.709281
432. PSNR : 24.370251, SSIM : 0.715365
433. PSNR : 22.300520, SSIM : 0.751581
434. PSNR : 23.337698, SSIM : 0.723799
435. PSNR : 19.012039, SSIM : 0.565265
436. PSNR : 28.777527, SSIM : 0.893441
437. PSNR : 25.299152, SSIM : 0.783641
438. PSNR : 26.647041, SSIM : 0.765577
439. PSNR : 26.204422, SSIM : 0.500681
440. PSNR : 27.755733, SSIM : 0.811915
441. PSNR : 36.602196, SSIM : 0.863794
442. PSNR : 22.466640, SSIM : 0.616273
443. PSNR : 27.157158, SSIM : 0.777477
444. PSNR : 26.808081, SSIM : 0.771902
445. PSNR : 22.673061, SSIM : 0.764996
446. PSNR : 21.002087, SSIM : 0.737065
447. PSNR : 31.310444, SSIM : 0.965121
448. PSNR : 32.215027, SSIM : 0.924592
449. PSNR : 20.723421, SSIM : 0.780869
450. PSNR : 26.670485, SS